In [1]:
!pip install pymysql sqlalchemy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.9 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
!pip install -q pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [13]:
data=pd.read_csv('/content/fraud_data.csv')
data.head()

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
0,04-01-2019 00:58,"""Stokes, Christiansen and Sipes""",grocery_net,14.37,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a3806e984cec6ac0096d8184c64ad3a1,65.654142,-164.722603,1
1,04-01-2019 15:06,Predovic Inc,shopping_net,966.11,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,a59185fe1b9ccf21323f581d7477573f,65.468863,-165.473127,1
2,04-01-2019 22:37,Wisozk and Sons,misc_pos,49.61,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,86ba3a888b42cd3925881fa34177b4e0,65.347667,-165.914542,1
3,04-01-2019 23:06,Murray-Smitham,grocery_pos,295.26,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,3a068fe1d856f0ecedbed33e4b5f4496,64.445035,-166.080207,1
4,04-01-2019 23:59,Friesen Lt,health_fitness,18.17,Wales,AK,64.7556,-165.6723,145,"""Administrator, education""",09-11-1939,891cdd1191028759dc20dc224347a0ff,65.447094,-165.446843,1


In [18]:
data['is_fraud'].unique()

array(['1', '1"2020-12-24 16:56:24"', '0', '0"2019-01-01 00:00:44"'],
      dtype=object)

In [19]:
data['is_fraud'] = pd.to_numeric(data['is_fraud'], errors='coerce')

# Drop rows where conversion failed
df = data.dropna(subset=['is_fraud'])

# Convert to int
df['is_fraud'] = df['is_fraud'].astype(int)

/tmp/ipython-input-19-2789604743.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_fraud'] = df['is_fraud'].astype(int)


**Total Fraud vs Non-Fraud Transactions**

In [20]:
q1 = """
SELECT is_fraud, COUNT(*) as total_transactions
FROM df
GROUP BY is_fraud
"""

In [21]:
pysqldf(q1)

,is_fraud,total_transactions
0,0,12600
1,1,1844


📉 Nearly 1 in 8 transactions (12.77%) is fraudulent, which is significantly high.

This suggests a high-risk environment, indicating the need for:

Enhanced fraud detection mechanisms

Real-time monitoring and alerts

Stronger authentication during transactions



 **Fraud Rate by Transaction Type**

In [25]:
q2 = """
SELECT
  COUNT(*) AS total_transactions,
  SUM(is_fraud) AS total_fraud_cases,
  ROUND(SUM(is_fraud) * 100.0 / COUNT(*), 2) AS fraud_rate_percent
FROM df;

"""
pysqldf(q2)

,total_transactions,total_fraud_cases,fraud_rate_percent
0,14444,1844,12.77


In [26]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], errors='coerce')
df['month'] = df['trans_date_trans_time'].dt.strftime('%Y-%m')

/tmp/ipython-input-26-2054847023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], errors='coerce')
/tmp/ipython-input-26-2054847023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['trans_date_trans_time'].dt.strftime('%Y-%m')


**Monthly Trend of Fraud vs Non-Fraud**

In [36]:
q3="""SELECT
  month,
  is_fraud,
  COUNT(*) AS total_transactions
FROM df
WHERE month IS NOT NULL
GROUP BY month, is_fraud
ORDER BY month, is_fraud;
 """

In [37]:
pysqldf(q3)

,month,is_fraud,total_transactions
0,2019-01,0,466
1,2019-01,1,41
2,2019-02,0,220
3,2019-02,1,19
4,2019-03,0,250
5,2019-03,1,20
6,2019-04,0,261
7,2019-04,1,35
8,2019-05,0,283
9,2019-05,1,57


Consistent Fraud Across Months:

Fraudulent transactions are recorded every month.

No month is fraud-free, highlighting the persistent risk.

📈 Peak Fraud Month:

May 2019 recorded 57 fraud cases, the highest in the timeline.

This coincides with a moderate number of total transactions, indicating a potential targeted fraud activity.

📉 Lowest Fraud Month:

September 2019 had only 8 fraud cases, lowest in 2019.

2020 Trends:

From Jan–Dec 2020, only fraud transactions are present (no non-fraud).

Could indicate filtered/simulated fraud dataset for 2020 or incomplete non-fraud records.

In [29]:
df['hour'] = df['trans_date_trans_time'].dt.hour

/tmp/ipython-input-29-3038483950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df['trans_date_trans_time'].dt.hour


**Fraud by Hour of Day**

In [40]:
q4="""SELECT
  CASE
    WHEN hour BETWEEN 5 AND 11 THEN 'Morning'
    WHEN hour BETWEEN 12 AND 16 THEN 'Afternoon'
    WHEN hour BETWEEN 17 AND 20 THEN 'Evening'
    WHEN hour BETWEEN 21 AND 23 THEN 'Night'
    ELSE 'Late Night'
  END AS hour_bin,
  is_fraud,
  COUNT(*) AS total_transactions
FROM df
WHERE hour IS NOT NULL
GROUP BY hour_bin, is_fraud
ORDER BY hour_bin, is_fraud;
"""

In [41]:
pysqldf(q4)

,hour_bin,is_fraud,total_transactions
0,Afternoon,0,920
1,Afternoon,1,26
2,Evening,0,740
3,Evening,1,33
4,Late Night,0,638
5,Late Night,1,228
6,Morning,0,826
7,Morning,1,33
8,Night,0,541
9,Night,1,371


 Night-Time Risk is Highest:

Night (00:00–03:59) has the highest fraud count: 371 frauds.

Indicates increased fraudulent activity when monitoring might be lower (possibly due to fewer system checks or human oversight).

🌃 Late Night (04:00–07:59):

Second highest fraud zone with 228 frauds.

Combines with Night to make 599 frauds during non-business hours, i.e., ~66% of all frauds.

🕛 Daytime (Morning + Afternoon + Evening):

Combined frauds: Morning: 33, Afternoon: 26, Evening: 33 → Total 92.

Despite high transaction volumes during the day, frauds are comparatively lower — suggesting more effective fraud detection in active hours.

**Fraud by State**

In [44]:
q5="""SELECT
  state,
  COUNT(*) AS total_transactions,
  SUM(is_fraud) AS fraud_cases,
  ROUND(SUM(is_fraud) * 100.0 / COUNT(*), 2) AS fraud_rate_percent
FROM df
GROUP BY state
ORDER BY fraud_rate_percent DESC; """

In [45]:
pysqldf(q5)

,state,total_transactions,fraud_cases,fraud_rate_percent
0,AK,173,65,37.57
1,NE,1460,238,16.30
2,OR,1211,197,16.27
3,CO,856,115,13.43
4,UT,597,73,12.23
5,CA,3374,410,12.15
6,NM,1003,121,12.06
7,MO,2329,267,11.46
8,WA,1150,126,10.96
9,WY,1100,119,10.82


 Alaska (AK) stands out with the highest fraud rate:

37.57% (65 frauds out of 173 transactions)

Indicates either a small sample size or targeted fraud activity — needs immediate investigation.

⚠️ High-Risk States (Fraud Rate > 10%):

NE (16.30%), OR (16.27%), CO (13.43%), UT (12.23%), CA (12.15%), NM (12.06%), MO (11.46%), WA (10.96%), WY (10.82%)

These states represent areas with above-average risk exposure.

📉 Lower Risk States:

ID (9.54%), AZ (9.51%), HI (9.30%)

Still significant, but relatively more secure compared to AK and NE.

AK requires special attention – either:

Increase transaction monitoring in Alaska, or

Check for data quality or outlier patterns (if sample size is small).

High-population states like CA have high fraud counts (410), which may represent both scale and systemic exposure.

This analysis can inform geographic fraud prevention strategies, such as:

Deploying state-wise risk models

Prioritizing fraud training or alerts in high-risk zones

**Fraud by Merchant Category**

In [48]:
q6="""SELECT
  category,
  COUNT(*) AS total_transactions,
  SUM(is_fraud) AS fraud_cases,
  ROUND(SUM(is_fraud) * 100.0 / COUNT(*), 2) AS fraud_rate_percent
FROM df
GROUP BY category
ORDER BY fraud_rate_percent DESC;
 """

In [49]:
pysqldf(q6)

,category,total_transactions,fraud_cases,fraud_rate_percent
0,shopping_net,1408,396,28.13
1,grocery_pos,1602,444,27.72
2,misc_net,821,223,27.16
3,shopping_pos,1354,194,14.33
4,gas_transport,1430,159,11.12
5,travel,384,34,8.85
6,misc_pos,823,64,7.78
7,grocery_net,474,32,6.75
8,entertainment,953,59,6.19
9,personal_care,990,57,5.76


Top Insights:
🚩 Highest Fraud Rates:

Shopping (Online):

shopping_net: 28.13% fraud rate (396 out of 1408)

Red flag: Online shopping is the most targeted.

Grocery (POS):

grocery_pos: 27.72% (444 frauds)

Unusual, possibly due to fake terminals or cloned cards.

Miscellaneous (Online):

misc_net: 27.16%

Likely due to less regulated/uncategorized e-commerce.

🛡️ Safer Categories (Fraud Rate < 10%):

home, health_fitness, food_dining, personal_care, kids_pets – All show lower risk (3–6%)

📈 Highest Fraud Counts:

Despite grocery_pos and shopping_net having similar rates, grocery_pos had the highest absolute fraud count: 444 cases.

Strategic Takeaways:
Online categories are highly vulnerable – prioritize:

Enhanced fraud detection for shopping_net, misc_net

Two-factor authentication for large online orders

In-person fraud exists too – especially grocery_pos and gas_transport.

Indicates possible terminal manipulation or lost/stolen cards

Use this segmentation to:

Train fraud models using category as a high-impact feature

Guide risk-based authentication strategies (e.g., extra checks for high-risk categories)

**Fraud by Job**

In [50]:
q7="""SELECT
  job,
  COUNT(*) AS total_transactions,
  SUM(is_fraud) AS fraud_cases
FROM df
GROUP BY job
ORDER BY fraud_cases DESC
LIMIT 10;
 """

In [51]:
pysqldf(q7)

,job,total_transactions,fraud_cases
0,"""Surveyor, minerals""",262,37
1,"""Administrator, education""",111,30
2,Colour technologist,111,29
3,Systems analyst,192,28
4,Network engineer,100,24
5,Naval architect,175,24
6,"""Surveyor, land/geomatics""",240,24
7,Research scientist (physical sciences),143,23
8,Human resources officer,119,23
9,Product designer,126,22


"Surveyor, minerals" has the highest fraud count (37) across 262 transactions – a 14.1% fraud rate.

"Administrator, education" and "Colour technologist" also show unusually high fraud volumes relative to their total transactions.

Technical roles such as Systems Analyst and Network Engineer show elevated fraud activity, possibly due to:

Higher-value transactions

Online activity frequency

Human resources officer and Product designer round out the top 10, indicating fraud is not limited to any one profession type.

In [52]:
df['amt_bucket'] = pd.cut(df['amt'], bins=[0, 50, 100, 200, 500, 1000, 2000, df['amt'].max()],
                          labels=['0-50', '51-100', '101-200', '201-500', '501-1000', '1001-2000', '2000+'])


/tmp/ipython-input-52-3904794615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amt_bucket'] = pd.cut(df['amt'], bins=[0, 50, 100, 200, 500, 1000, 2000, df['amt'].max()],


**Fraud by Amount Buckets**

In [53]:
q8="""SELECT
  amt_bucket,
  is_fraud,
  COUNT(*) AS total_transactions
FROM df
GROUP BY amt_bucket, is_fraud
ORDER BY amt_bucket, is_fraud;
 """

In [54]:
pysqldf(q8)

,amt_bucket,is_fraud,total_transactions
0,0-50,0,6662
1,0-50,1,406
2,1001-2000,0,26
3,1001-2000,1,227
4,101-200,0,1661
5,101-200,1,39
6,2000+,0,5
7,201-500,0,451
8,201-500,1,513
9,501-1000,0,89


Fraud is concentrated in medium to high-value ranges (₹201–₹2000).

Low-value transactions (< ₹100) are less risky and more frequent.

Fraudsters appear to prefer high-reward targets, which aligns with typical fraud behavior (maximize gain in a single shot).

| Amount Bucket | Fraud Rate Insight                                                                     |
| ------------- | -------------------------------------------------------------------------------------- |
| **0–50**      | Very high volume (6,662), **low fraud risk** (406 cases – \~6%) – mostly genuine       |
| **51–100**    | Moderate volume, **very low fraud cases** – considered low risk                        |
| **101–200**   | \~2.3% fraud rate – still fairly safe                                                  |
| **201–500**   | 🔥 **High fraud activity** – 513 frauds out of 964 transactions ⇒ **\~53% fraud rate** |
| **501–1000**  | 🔥 Very risky – **\~88% fraud rate** (642 out of 731)                                  |
| **1001–2000** | ⚠️ Severe risk – **\~90% fraud rate**                                                  |
| **2000+**     | Minimal data, but still some legitimate transactions present                           |


Recommendations
Amount buckets should be used as primary features in fraud prediction models.

Implement threshold-based risk triggers:

Transactions > ₹200 ⇒ require multi-factor authentication (MFA) or OTP confirmation.

Transactions > ₹1000 ⇒ route to manual review or dynamic fraud scoring.

Use this data to design spending alerts and smart caps for user-configured fraud protection.

💳 Transaction Amount-Based Risk Triggers
Fraud is highly concentrated in mid-to-high value ranges ($201–$2000).

Recommendations:

Set transaction amount thresholds:

$201–$1000: Trigger OTP, location verification, or biometric checks.

$1000+: Send for AI-based scoring or human review.

Flag large transactions that deviate from user's historical average.

2. 🕒 Time-of-Day Risk Windowing
Peak fraud activity is during:

Late Night (12AM–3AM)

Night (9PM–12AM)

Recommendations:

Impose transaction cooldowns or stricter validations during these hours.

Create time-aware fraud risk models to capture behavioral shifts.

3. 🏷️ High-Risk Merchant Categories
Categories with highest fraud rates:

shopping_net – 28.13%

grocery_pos – 27.72%

misc_net – 27.16%

Recommendations:

Implement two-factor authentication for online and POS merchant types.

Strengthen monitoring around non-branded, high-ticket online orders.

4. 🌍 Geographic Risk Zones
Top 3 high-risk states by fraud rate:

Alaska (37.6%)

Nebraska (16.3%)

Oregon (16.27%)

Recommendations:

Use state-level fraud scoring.

Integrate device geo-location and IP checks.

5. 👥 Occupation-Based Risk Patterns
Jobs with higher fraud rates include:

"Surveyor, minerals", "Administrator, education", "Systems Analyst"

Recommendations:

Consider profession as a categorical variable in model training.

Apply segmentation-based fraud limits where appropriate.

6. 📊 Overall Fraud Snapshot
Total transactions: ~14,444

Fraudulent: 1,844 → ~12.75% overall fraud rate

Not uniform – driven by amounts, times, locations, categories, and user profiles.
